In [ ]:
!pip install -q -U kaggle_environments

In [ ]:
from kaggle_environments import make

# Создаем окружение для игры
env = make("rps", configuration={"episodeSteps": 100})

# Агент, который всегда выбирает "камень"
def rock_agent(observation, configuration):
    return 0  # 0 означает "камень"

# Агент, который делает случайный выбор
import random
def random_agent(observation, configuration):
    return random.randint(0, 2)  # 0, 1, 2: камень, бумага, ножницы

# Запуск игры между двумя агентами
env.run([rock_agent, random_agent])

# Визуализация результатов
env.render(mode="ipython")

**Разработка собственных агентов - ботов (15 стратегий)**



In [ ]:
# 1. Агент, который всегда выбирает "камень"
def rock_agent(observation, configuration):
    return 0  # Камень

# 2. Агент, который всегда выбирает "ножницы"
def scissors_agent(observation, configuration):
    return 2  # Ножницы

# 3. Агент, который всегда выбирает "бумагу"
def paper_agent(observation, configuration):
    return 1  # Бумага

# 4. Агент, который делает случайный выбор
def random_agent(observation, configuration):
    return random.randint(0, 2)  # Случайный выбор: камень, бумага, ножницы

# 5. Агент, который зеркалирует предыдущий ход противника
def mirror_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)  # В первый ход случайный выбор
    else:
        return observation.lastOpponentAction  # Повторяет ход противника

# 6. Агент, который всегда контратакует прошлый ход противника
def counter_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        # Всегда выбирает ход, который побеждает предыдущий ход противника
        return (observation.lastOpponentAction + 1) % 3

# 7. Агент, который чаще выбирает камень, но иногда меняет
def biased_random_agent(observation, configuration):
    return random.choices([0, 1, 2], weights=[0.6, 0.2, 0.2])[0]  # Чаще выбирает камень

# 8. Агент, который чередует ходы камень, бумага, ножницы
def pattern_agent(observation, configuration):
    return observation.step % 3  # Камень -> Бумага -> Ножницы

# 9. Агент, который повторяет первый ход противника
def copy_first_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        return observation['actionHistories'][1][0]  # Копирует первый ход противника

# 10. Агент, который меняет стратегию при проигрыше
def adaptive_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        if observation.reward < 0:  # Проигрыш
            return (observation.lastOpponentAction + 1) % 3  # Меняем на другой ход
        else:
            return observation.lastOpponentAction  # Если выиграл, повторяет

# 11. Агент, который дважды выбирает камень, а затем случайно
def rock_twice_then_random_agent(observation, configuration):
    if observation.step < 2:
        return 0  # Первые два хода "камень"
    return random.randint(0, 2)  # Дальше случайный выбор

# 12. Агент, который повторяет выигрышный ход
def repeat_winner_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    if observation.reward > 0:  # Если выиграл, повторяет ход
        return observation.lastOpponentAction
    else:
        return random.randint(0, 2)  # Если проиграл, случайный выбор

# 13. Агент, который использует последние два хода для анализа
def last_two_moves_agent(observation, configuration):
    if observation.step < 2:
        return random.randint(0, 2)
    # Анализируем последние два хода противника
    last_two = [observation['actionHistories'][1][-1], observation['actionHistories'][1][-2]]
    return (sum(last_two) % 3)  # Как-то обрабатываем последние два хода

# 14. Агент, который чередует камень и ножницы
def alternate_rock_scissors_agent(observation, configuration):
    if observation.step % 2 == 0:
        return 0  # Камень
    else:
        return 2  # Ножницы

# 15. Агент, который копирует предыдущий ход противника, но чередует с камнем
def tit_for_tat_agent(observation, configuration):
    if observation.step == 0:
        return 0
    else:
        return (observation.lastOpponentAction + observation.step) % 3

In [ ]:
# Список агентов для турнира
agents = [
    rock_agent, scissors_agent, paper_agent, random_agent,
    mirror_agent, counter_agent, biased_random_agent,
    pattern_agent, copy_first_agent, adaptive_agent,
    rock_twice_then_random_agent, repeat_winner_agent,
    last_two_moves_agent, alternate_rock_scissors_agent,
    tit_for_tat_agent
]


**Запуск турнира между агентами**

In [ ]:
# Запуск турнира: каждый агент играет с каждым
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        # Запуск игры между двумя агентами
        env.reset()
        env.run([agents[i], agents[j]])
        print(f"Игра {i+1} (Agent {i+1} против Agent {j+1})")

        # Визуализируем результаты каждой игры
        env.render(mode="ipython")


**"Камень-ножницы-бумага" - игра между простыми ботами и ML-ботом**

In [ ]:
# Импортируем необходимые библиотеки
import random
from collections import Counter

# Простые агенты: всегда выбирают камень, ножницы или бумагу
def rock_agent(observation, configuration):
    return 0  # Всегда выбирает камень

def scissors_agent(observation, configuration):
    return 2  # Всегда выбирает ножницы

def paper_agent(observation, configuration):
    return 1  # Всегда выбирает бумагу

# Агент, который делает случайный выбор
def random_agent(observation, configuration):
    return random.randint(0, 2)

# Простой ML-агент, который отслеживает частоту ходов противника и использует это для выбора
class SimpleMLAgent:
    def __init__(self):
        self.opponent_actions = []  # Хранение ходов противника

    def __call__(self, observation, configuration):
        if observation.step > 0:
            # Добавляем последний ход противника в список
            self.opponent_actions.append(observation.lastOpponentAction)

        if len(self.opponent_actions) == 0:
            # Если это первый ход, выбираем случайно
            return random.randint(0, 2)

        # Анализируем частоту ходов противника
        action_counts = Counter(self.opponent_actions)
        most_common_action = action_counts.most_common(1)[0][0]  # Самый частый ход противника

        # Возвращаем ход, который победит самый частый ход противника
        return (most_common_action + 1) % 3

# Создаем агента
ml_agent = SimpleMLAgent()

# Пример запуска игры между ML-агентом и случайным агентом
from kaggle_environments import make

# Создаем окружение для игры
env = make("rps", configuration={"episodeSteps": 100})

# Запуск игры между двумя агентами
env.run([ml_agent, random_agent])

# Визуализация результатов
env.render(mode="ipython")



**Визуализация: "Камень-ножницы-бумага" - Простые боты + ML-бот (pandas, seaborn, matplotlib)**

In [ ]:

import random
from collections import Counter
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Простые агенты и ML-агент
def rock_agent(observation, configuration):
    return 0

def random_agent(observation, configuration):
    return random.randint(0, 2)

class SimpleMLAgent:
    def __init__(self):
        self.opponent_actions = []

    def __call__(self, observation, configuration):
        if observation.step > 0:
            self.opponent_actions.append(observation.lastOpponentAction)

        if len(self.opponent_actions) == 0:
            return random.randint(0, 2)

        action_counts = Counter(self.opponent_actions)
        most_common_action = action_counts.most_common(1)[0][0]
        return (most_common_action + 1) % 3

ml_agent = SimpleMLAgent()

# Создаем окружение для игры
from kaggle_environments import make
env = make("rps", configuration={"episodeSteps": 100})

# Список для хранения выборов каждого агента
choices = []

# Запуск нескольких игр между агентами и сбор данных о выборе
for _ in range(50):  # 50 игр для примера
    env.reset()
    game = env.run([ml_agent, random_agent])

    for step in game:
        ml_choice = step[0]['action']  # Выбор ML-агента
        random_choice = step[1]['action']  # Выбор случайного агента
        choices.append({'Agent': 'ML Agent', 'Choice': ml_choice})
        choices.append({'Agent': 'Random Agent', 'Choice': random_choice})

# Преобразуем данные в DataFrame для визуализации
df_choices = pd.DataFrame(choices)

# Заменим числовые значения действий на их текстовые обозначения
df_choices['Choice'] = df_choices['Choice'].replace({0: 'Rock', 1: 'Paper', 2: 'Scissors'})

# Визуализация распределения выборов каждого агента
plt.figure(figsize=(10, 6))
sns.countplot(x='Choice', hue='Agent', data=df_choices)
plt.title('Распределение выборов действий (камень, ножницы, бумага)')
plt.show()


# Сбор данных: Во время каждой игры мы собираем данные о том, какое действие (камень, бумага, ножницы) выбрал каждый агент на каждом шаге.
# Визуализация: Строим столбчатую диаграмму, показывающую, как часто каждый агент выбирал камень, бумагу или ножницы
# Это позволит увидеть, предпочитают ли агенты определённые действия и насколько они отличаются в стратегиях.